In [5]:
!pip install fastapi uvicorn pyngrok python-multipart pillow nest-asyncio
!pip install python-dotenv


In [6]:
# Import necessary libraries
import nest_asyncio
import uvicorn
from fastapi import FastAPI, File, UploadFile, Form
from fastapi.responses import StreamingResponse, JSONResponse
from pyngrok import ngrok
import cv2
import numpy as np
from io import BytesIO
from PIL import Image
import asyncio

# Apply nest_asyncio to allow running FastAPI in Jupyter
nest_asyncio.apply()

# Initialize FastAPI app
app = FastAPI()

# Health check endpoint
@app.get("/health")
async def health_check():
    return JSONResponse(content={"status": "healthy"}, status_code=200)


def process_image(image: np.ndarray, prompt: str) -> np.ndarray:
    # Copy image to avoid modifying original

    # random rotate image
    angle = np.random.randint(0, 360)
    h, w = image.shape[:2]
    center = (w // 2, h // 2)
    rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated_image = cv2.warpAffine(image, rotation_matrix, (w, h))


    image_with_text = rotated_image.copy()
    
    # Set font parameters
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 1
    thickness = 2
    color = (255, 0, 0)  # Red text in RGB (will look blue if not converted to RGB)

    # Put the text at the top-left corner
    cv2.putText(image_with_text, prompt, (10, 30), font, font_scale, color, thickness, cv2.LINE_AA)

    return image_with_text

# FastAPI endpoint to handle image upload and prompt
@app.post("/process-image/")
async def process_image_endpoint(file: UploadFile = File(...), prompt: str = Form(...)):
    # Read the uploaded image
    image_data = await file.read()
    image = Image.open(BytesIO(image_data)).convert("RGB")
    image_np = np.array(image)

    # Process the image based on the prompt
    processed_image_np = process_image(image_np, prompt)

    # Convert processed image back to bytes
    processed_image_pil = Image.fromarray(processed_image_np)
    output_buffer = BytesIO()
    processed_image_pil.save(output_buffer, format="PNG")
    output_buffer.seek(0)

    # Return the processed image as a streaming response
    return StreamingResponse(output_buffer, media_type="image/png")




In [7]:


# Function to start ngrok and FastAPI server
async def start_server(Token_ngrok):
    # Set your ngrok authtoken
    ngrok.set_auth_token(Token_ngrok)  # Replace with your ngrok authtoken

    # Start ngrok tunnel
    public_url = ngrok.connect(8000, bind_tls=True).public_url
    print(f"ngrok public URL: {public_url}")

    # Configure and run the FastAPI server
    config = uvicorn.Config(app, host="0.0.0.0", port=8000)
    server = uvicorn.Server(config)
    await server.serve()

In [ ]:
from dotenv import load_dotenv
import os
from langchain_openai import ChatOpenAI

# Load environment variables from .env file
load_dotenv()

# Use the API key from the environment
# Token_ngrok = "xxxxx"

Token_ngrok= os.getenv("NGROK_TOKEN_KEY")

asyncio.run(start_server(Token_ngrok ))

INFO:     Started server process [39596]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


ngrok public URL: https://7a39-2001-fb1-c1-4d4c-adf2-ed0f-884a-442f.ngrok-free.app
INFO:     2001:fb1:c1:4d4c:adf2:ed0f:884a:442f:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     2001:fb1:c1:4d4c:adf2:ed0f:884a:442f:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     20.218.238.122:0 - "POST /process-image HTTP/1.1" 307 Temporary Redirect
INFO:     20.218.238.122:0 - "POST /process-image/ HTTP/1.1" 200 OK
